# **Azure AutoML**


### **1. Introducción a Azure AutoML**
Azure AutoML es una herramienta dentro de Azure Machine Learning que automatiza el proceso de selección de modelos, ajuste de hiperparámetros y validación cruzada. Está diseñado para acelerar el desarrollo de modelos de machine learning, especialmente para usuarios sin experiencia avanzada en ciencia de datos.

#### **1.1. ¿Qué es AutoML?**
- **Automatización del Ciclo de Vida del Modelo:**  
  - Automatiza tareas como la selección de algoritmos, preprocesamiento de datos y optimización de hiperparámetros.
- **Casos de Uso Comunes:**  
  - Clasificación, regresión, pronóstico y detección de anomalías.

#### **1.2. Arquitectura de AutoML**
- **Experimentos:**  
  - Un experimento es una colección de ejecuciones (runs) que ejecutan diferentes configuraciones de modelos.
- **Compute Targets:**  
  - Recursos computacionales donde se ejecutan las pruebas de AutoML.
- **Resultados:**  
  - Genera un informe detallado con métricas de rendimiento y recomendaciones de modelos.

#### **Referencias:**
- [Azure AutoML Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/concept-automated-ml)

---

### **2. Configuración Inicial**
Para comenzar a usar Azure AutoML, es necesario configurar el entorno.

#### **2.1. Instalación del SDK**
Instala el SDK de Azure ML:
```bash
pip install azureml-core azureml-train-automl
```

#### **2.2. Autenticación**
Azure AutoML utiliza autenticación basada en tokens. Puedes usar `config.json` o Managed Identities para conectarte al área de trabajo.

#### **Referencias:**
- [Azure ML Authentication Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication)

---



## **Módulo 2: Proceso de AutoML (4 horas)**  
**Objetivo:** Aprender cómo funciona el proceso de AutoML y cómo configurarlo para diferentes tipos de problemas.

---

### **1. Preparación de Datos**
AutoML requiere datos limpios y bien estructurados para funcionar correctamente.

#### **1.1. Conjuntos de Datos Registrados**
- Los conjuntos de datos deben estar registrados en Azure ML para ser utilizados en AutoML.

#### **1.2. Requisitos de Datos**
- **Columna de Etiqueta:**  
  - La columna objetivo debe estar claramente definida.
- **Formato de Datos:**  
  - Soporta formatos como CSV, Parquet y tabulares.

#### **Ejemplo Práctico: Registro de un Conjunto de Datos**
```python
from azureml.core import Dataset

datastore = ws.get_default_datastore()
datastore.upload(src_dir='./datos', target_path='datos')

dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'datos/mi-dataset.csv'))
dataset.register(workspace=ws, name='mi-dataset')
```

#### **Referencias:**
- [Azure ML Datasets Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets)

---

### **2. Configuración de AutoML**
La configuración de AutoML depende del tipo de problema que estés abordando.

#### **2.1. Tipos de Tareas**
- **Clasificación:**  
  - Ejemplo: Predicción de categorías como "spam/no spam".
- **Regresión:**  
  - Ejemplo: Pronóstico de valores numéricos como precios de vivienda.
- **Pronóstico:**  
  - Ejemplo: Predicción de series temporales como ventas futuras.
- **Detección de Anomalías:**  
  - Ejemplo: Identificación de transacciones fraudulentas.

#### **2.2. Parámetros de Configuración**
- **Task:** Tipo de tarea (clasificación, regresión, etc.).
- **Primary Metric:** Métrica principal para evaluar el rendimiento (precisión, RMSE, etc.).
- **Iterations:** Número máximo de iteraciones.
- **Compute Target:** Recursos computacionales donde se ejecuta AutoML.

#### **Referencias:**
- [AutoML Configuration Parameters](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train)

---

### **3. Ejecución de AutoML**
Una vez configurado, puedes enviar un experimento de AutoML para que comience el proceso de entrenamiento.

#### **Ejemplo Práctico: Configuración y Ejecución de AutoML**
```python
from azureml.train.automl import AutoMLConfig
from azureml.core import Experiment

automl_config = AutoMLConfig(
    task="classification",
    primary_metric="accuracy",
    compute_target="mi-cluster",
    training_data=dataset,
    label_column_name="etiqueta",
    iterations=10
)

experimento = Experiment(workspace=ws, name="mi-experimento-automl")
corrida = experimento.submit(automl_config)
corrida.wait_for_completion(show_output=True)
```

#### **Referencias:**
- [Running AutoML Experiments](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-automl)

---

## **Módulo 3: Evaluación y Selección de Modelos (3 horas)**  
**Objetivo:** Aprender a interpretar los resultados de AutoML y seleccionar el mejor modelo.

---

### **1. Resultados de AutoML**
AutoML genera un informe detallado con métricas de rendimiento y recomendaciones de modelos.

#### **1.1. Métricas Clave**
- **Clasificación:** Precisión, recall, F1-score, ROC-AUC.
- **Regresión:** RMSE, MAE, R².
- **Pronóstico:** SMAPE, MASE.

#### **1.2. Visualización de Resultados**
- Azure ML proporciona gráficos interactivos para analizar el rendimiento de los modelos.

#### **Ejemplo Práctico: Visualización de Resultados**
```python
best_run, best_model = corrida.get_output()
print(f"Mejor modelo: {best_model}")
print(f"Mejor ejecución: {best_run.id}")
```

#### **Referencias:**
- [Interpreting AutoML Results](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-understand-automl-results)

---

### **2. Selección del Mejor Modelo**
AutoML selecciona automáticamente el mejor modelo basado en la métrica principal. Sin embargo, puedes explorar otros modelos si es necesario.

#### **Ejemplo Práctico: Exploración de Modelos**
```python
children = list(corrida.get_children())
metrics_list = {}

for child in children:
    metrics = child.get_metrics()
    metrics_list[child.id] = metrics

# Mostrar métricas de todos los modelos
for run_id, metrics in metrics_list.items():
    print(f"Run ID: {run_id}, Metrics: {metrics}")
```

---

## **Módulo 4: Despliegue y Monitoreo de Modelos (3 horas)**  
**Objetivo:** Aprender a desplegar y monitorear modelos generados por AutoML.

---

### **1. Despliegue del Modelo**
El modelo generado por AutoML puede desplegarse como un servicio web en Azure Container Instances (ACI) o Azure Kubernetes Service (AKS).

#### **1.1. Registro del Modelo**
- El mejor modelo se registra automáticamente en el Model Registry.

#### **1.2. Creación de Endpoints**
- Exposición del modelo como una API RESTful.

#### **Ejemplo Práctico: Despliegue en ACI**
```python
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py")
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name="mi-servicio",
                       models=[best_model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)
print(f"Servicio desplegado: {service.scoring_uri}")
```

#### **Referencias:**
- [Deploying AutoML Models](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-automl-models)

---

### **2. Monitoreo del Modelo**
Azure ML proporciona herramientas para monitorear el rendimiento de los modelos en producción.

#### **2.1. Métricas de Rendimiento**
- Seguimiento de métricas como precisión, latencia y tráfico.

#### **2.2. Alertas y Rollbacks**
- Configuración de alertas para detectar problemas y realizar rollbacks si es necesario.

#### **Referencias:**
- [Monitoring AutoML Models](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-monitor-models)

---

## **Conclusión**
Este curso proporciona una visión completa de Azure AutoML, desde la configuración inicial hasta el monitoreo de modelos en producción. Los participantes estarán preparados para implementar soluciones de machine learning automatizadas y escalables en entornos reales.

#### **Referencias Generales:**
- [Azure AutoML Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/concept-automated-ml)
- [Scikit-learn Documentation](https://scikit-learn.org/stable/documentation.html)